## Installation
-  Run this only once

In [ ]:
!pip install gdown
!pip install tf_slim
!pip install numpy==1.22
!pip install LibRecommender


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20443 sha256=e64e9b3685e846daed3737d3ecc9f61a519665688bd73ce2ac3d8f586bb6f064
  Stored in directory: /root/.cache/pip/wheels/6a/65/c4/d27eeee9ba3fc150a0dae150519591103b9e0dbffde3ae77dc
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3530 sha256=2b86a3e2aebad1bb92771d69221f7c37270348e78fa14e366c6317fd3585b10a
  Stored in directory: /root/.cache/pip/wheels/ba/d9/a0/ddd93af16d5855dd9bad417623e70948fdac119d1d34fb17c8
Successfully built fst-pso miniful
  Attempting uninstall: gen

## Restart 

In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Loading

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np
from datetime import datetime

In [ ]:
TIMESTAMP=None

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1tZlrPeJf7geYfRRTxeRhdPU_Hq5m8tg6

Downloading...
From: https://drive.google.com/uc?id=1tZlrPeJf7geYfRRTxeRhdPU_Hq5m8tg6
To: /content/Travel_TripAdvisor_v1.zip
100% 155k/155k [00:00<00:00, 111MB/s]


In [ ]:
!unzip "Travel_TripAdvisor_v1.zip"  -d  "/content"

Archive:  Travel_TripAdvisor_v1.zip
   creating: /content/Travel_TripAdvisor_v1/
  inflating: /content/Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls  
  inflating: /content/Travel_TripAdvisor_v1/ReadMe.html  


In [ ]:
!pip install pandas openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import pandas as pd

xls = pd.ExcelFile(r"Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls") 

rating_df = xls.parse(0) #2 is the sheet number+1 thus if the file has only 1 sheet write 0 in paranthesis



In [ ]:
rating_df=rating_df.reset_index(drop=True)

In [ ]:

rating_df.rename(columns = {'ID_USER':'user'}, inplace = True)
rating_df.rename(columns = {'ID_HOTEL':'item'}, inplace = True)
rating_df.rename(columns = {'Rating':'label'}, inplace = True)

## Name of the file where user item interaction data avaiable?
- Column name should be in following name and order
- [ **itemID, userID , rating , [other features]** ]


### Dense columns

In [ ]:
dense_col=None

### Sparse columns

In [ ]:
col_sparse= ['USER_STATE', 'USER_TIMEZONE','Trip Type']


### User contexual features other than "user" column

In [ ]:

user_col = ['USER_STATE', 'USER_TIMEZONE','Trip Type']


### User contexual features other than "item" column

In [ ]:
item_col=None


### Run below cell

In [ ]:
allcols=['user','item','label']
if user_col is not None:
  for i in user_col:
    allcols.append(i)
if item_col is not None:
  for i in item_col:
    allcols.append(i)

In [ ]:
listcols = []
[listcols.append(x) for x in allcols if x not in listcols]

[None, None, None, None, None, None]

In [ ]:
listcols

['user', 'item', 'label', 'USER_STATE', 'USER_TIMEZONE', 'Trip Type']

In [ ]:
rating_df=rating_df[listcols]

##### Code Setup

In [ ]:
# Evaluation metrics
import libreco
import tensorflow as tf
from libreco.data import DatasetFeat, DataInfo
from libreco.data import  split_by_ratio
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats
import torch
from libreco.algorithms import (
    FM,
    WideDeep,
    DeepFM
)

import time
import pandas as pd

metrics = [
    "precision",
    "recall",
    "map",
    "ndcg"
]

# Call Librecommender API
# datasettype ="pure" or "feat"
def run(rating_df,user_col, item_col, col_sparse, dense_col,datasettype="pure", algo=["als"],hyperparam={"als":{ "embed_size":16,
        "n_epochs":10,
        "reg":5.0,
        "alpha":10,
        "use_cg":False,
        "n_threads":1,
        "seed":42}}, neg_samples=True):
    train_data, test_data, data_info=None,None,None
    tf.compat.v1.reset_default_graph()
    train_data, test_data, data_info = preprocessingFeature(rating_df,user_col, item_col, col_sparse, dense_col, neg_samples) 
    models = dict()
    evals = dict()
    if(neg_samples):
      train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
      test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    for i in algo:
        if i == 'WideDeep':
            
            als, eval, algo_data_info = WideDeep(train_data, test_data, data_info,hyperparam["WideDeep"])
            models['WideDeep'] = als
            evals['WideDeep'] = eval
            print("Evaluation Results for WideDeep:")
            print(evals['WideDeep'])
        
       
          
    return models, evals

# data preprocessing
def preprocessingFeature(rating_df,user_col, item_col, col_sparse, dense_col, neg_samples=True):
    data = rating_df.copy()

    train, test = split_by_ratio(data, test_size=0.2)
    train_data, data_info = DatasetFeat.build_trainset(
        train, user_col, item_col, col_sparse, dense_col,
        shuffle=False,
    )
    test_data = DatasetFeat.build_testset(test, shuffle=False)
    if neg_samples:
        train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
        test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    return train_data, test_data, data_info


# Recommend items to user
def recommend(model, userid, k=10):
    return model.recommend_user(user=userid, n_rec=k)

# Find similar items
def similarItems(model, itemids, k=10):
    return model.search_knn_items(item=itemids, k=k)


# Model : Wide and deep
def WideDeep(train_data, test_data, data_info,hyperparam={"embed_size":16,
        "n_epochs":10,
        "lr":{"wide": 0.01, "deep": 1e-4},
        "lr_decay":False,
        "reg":None,
        "batch_size":256,
        "num_neg":1,
        "use_bn":False,
        "dropout_rate":None,
        "hidden_units":"128,64,32",
        "tf_sess_config":None,}):
    model = libreco.algorithms.WideDeep(
        "ranking",
        data_info,
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],  
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        use_bn=hyperparam["use_bn"],
        dropout_rate=hyperparam["dropout_rate"],
        hidden_units=hyperparam["hidden_units"],
        tf_sess_config=hyperparam["tf_sess_config"],
    )
    model.fit(
        train_data,
        verbose=2,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=model, data=test_data, metrics=metrics)
    return model, result, data_info

    


Instructions for updating:
non-resource variables are not supported in the long term


## Run wide and deep

In [ ]:

models, evals = run(rating_df,user_col, item_col, col_sparse, dense_col, algo=["WideDeep"], hyperparam={"WideDeep":{"embed_size":16,
        "n_epochs":10,
        "lr":{"wide": 0.01, "deep": 1e-4},
        "lr_decay":False,
        "reg":None,
        "batch_size":256,
        "num_neg":1,
        "use_bn":False,
        "dropout_rate":None,
        "hidden_units":"128,64,32",
        "tf_sess_config":None,} },neg_samples=True)


random neg item sampling elapsed: 0.004s
random neg item sampling elapsed: 0.001s
random neg item sampling elapsed: 0.009s
random neg item sampling elapsed: 0.001s
total params: 72,848 | embedding params: 52,331 | network params: 20,517
Training start time: 2023-02-07 21:57:01


train: 100%|██████████| 65/65 [00:03<00:00, 17.94it/s]


Epoch 1 elapsed: 3.635s
	 train_loss: 0.6933


eval_listwise: 100%|██████████| 23/23 [00:00<00:00, 170.39it/s]


	 eval precision@10: 0.0015
	 eval recall@10: 0.0134
	 eval map@10: 0.0030
	 eval ndcg@10: 0.0057


...

Epoch 10 elapsed: 0.339s
	 train_loss: 0.6772


eval_listwise: 100%|██████████| 23/23 [00:00<00:00, 180.47it/s]


	 eval precision@10: 0.0017
	 eval recall@10: 0.0146
	 eval map@10: 0.0095
	 eval ndcg@10: 0.0112


eval_listwise: 100%|██████████| 103/103 [00:00<00:00, 339.21it/s]

Evaluation Results for WideDeep:
{'precision': 0.0017031630170316304, 'recall': 0.014598540145985401, 'map': 0.009468775344687753, 'ndcg': 0.011203235287356731}


#### Recommendations to users

In [ ]:
bpr=models['WideDeep']

In [ ]:

results= recommend(bpr,1, 10)
print(results)

Detect unknown user: 1
{1: array([  94675,  295455, 1845691,  101910,  224185,  224185,   84320,
        223714,   91979,   84480])}
